# ucsc1996

See [GitHub Issue](https://github.com/krank-sources/dreambank/issues/7).

In [ ]:
from datetime import datetime, timezone
import os

import pandas as pd
import pooch

In [ ]:
DATASETS = ["ucsc_women"]

## Load

In [ ]:
# Retrieve the latest datasets from GitHub releases
# OR from local file if during development of a new release.
DREAMBANK_VERSION = "v1.0.0-alpha5"
datasets_kwargs = {
    "url": f"https://github.com/krank-sources/dreambank/releases/download/{DREAMBANK_VERSION}/datasets.csv.xz",
    "known_hash": "md5:1475582e2daa1da53920df50cb9fc98e",
}
dreams_kwargs = {
    "url": f"https://github.com/krank-sources/dreambank/releases/download/{DREAMBANK_VERSION}/dreams.csv.xz",
    "known_hash": "md5:2dcab92f9d9515df174388babb5c9e5a",
    "progressbar": True,
}

datasets_fname = pooch.retrieve(**datasets_kwargs)
dreams_fname = pooch.retrieve(**dreams_kwargs)
# datasets_fname = "../output/datasets.csv.xz"
# dreams_fname = "../output/dreams.csv.xz"

In [ ]:
datasets = pd.read_csv(datasets_fname)
dreams = pd.read_csv(dreams_fname)

## Process

In [ ]:
dreams = dreams[dreams["dataset"].isin(DATASETS)].copy()

In [ ]:
# Verify assumptions about the data
assert dreams["metadata"].str.contains("/female").all()
# Extract age from metadata (format: "XX/female")
ages = dreams["metadata"].str.extract(r'(\d+)/')[0].astype(int)
assert (ages >= 18).all() and (ages <= 24).all(), "All ages should be between 18-24"

In [ ]:
# Create constant sex column
dreams["sex"] = "female"
# Create author column by extracting digits from dream_id (keep as string)
dreams["author"] = dreams["dream_id"].str.extract(r'(\d+)')[0].astype(str)
# Create age column by extracting from metadata
dreams["age"] = dreams["metadata"].str.extract(r'(\d+)/')[0].astype(int)

In [ ]:
dreams = dreams.rename(columns={"dream_text": "report"})
dreams = dreams.reindex(columns=["author", "age", "sex", "report"])
dreams = dreams.sort_values("author")

In [ ]:
# Minor fix: See https://github.com/krank-sources/dreambank/issues/6
dreams["report"] = dreams["report"].str.replace(r"\\$", "", regex=True)

In [ ]:
dreams.head()

In [ ]:
dreams.describe()

In [ ]:
dreams.info(memory_usage="deep")

## Export

In [ ]:
assert not dreams.isna().any(axis=None)
assert not dreams.duplicated().any()
assert not dreams.duplicated(subset="report").any()
assert dreams["sex"].isin(["female", "male"]).all()
assert (dreams["report"] == dreams["report"].str.strip()).all()

In [ ]:
outpath = "./output/ucsc1996.csv"
os.makedirs(os.path.dirname(outpath), exist_ok=True)

TO_CSV_KWARGS = {
    "index": False,
    "na_rep": "N/A",
    "sep": ",",
    "mode": "x",  # Switch to `w` to overwrite existing file
    "encoding": "utf-8-sig",  # Include sig/BOM for better compatibility with Excel
    "lineterminator": "\n",
    "quoting": 2,  # 2 = csv.QUOTE_NONNUMERIC
    "quotechar": '"',
    "doublequote": True,
}
dreams.to_csv(outpath, **TO_CSV_KWARGS)

print(f"file: {os.path.basename(outpath)}")
print(f"size: {os.path.getsize(outpath) / 1e6} MB")
print(f"md5: {pooch.file_hash(outpath, alg='md5')}")
print(f"sha256: {pooch.file_hash(outpath, alg='sha256')}")
print(f"timestamp: {datetime.fromtimestamp(os.path.getmtime(outpath), tz=timezone.utc).isoformat(timespec='seconds')}")